In [3]:
import pandas as pd

You have been provided with a dataset of over 350,000 such complaints for 5 common issue types. Your goal is to train a text classification model to identify the issue type based on the consumer complaint narrative. The data can be downloaded from

In [5]:
corpus = pd.read_csv('../data/complaints.csv')

In [6]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353432 entries, 0 to 353431
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Consumer complaint narrative  353432 non-null  object
 1   Issue                         353432 non-null  object
dtypes: object(2)
memory usage: 5.4+ MB
